# CLIP Interrogator 2.4 by [@pharmapsychotic](https://twitter.com/pharmapsychotic)

Want to figure out what a good prompt might be to create new images like an existing one? The CLIP Interrogator is here to get you answers!

<br>

For Stable Diffusion 1.X choose the **ViT-L** model and for Stable Diffusion 2.0+ choose the **ViT-H** CLIP Model.

This version is specialized for producing nice prompts for use with Stable Diffusion and achieves higher alignment between generated text prompt and source image. You can try out the old [version 1](https://colab.research.google.com/github/pharmapsychotic/clip-interrogator/blob/v1/clip_interrogator.ipynb) to see how different CLIP models ranks terms.

You can also run this on HuggingFace and Replicate<br>
[![Generic badge](https://img.shields.io/badge/🤗-Open%20in%20Spaces-blue.svg)](https://huggingface.co/spaces/pharma/CLIP-Interrogator) [![Replicate](https://replicate.com/pharmapsychotic/clip-interrogator/badge)](https://replicate.com/pharmapsychotic/clip-interrogator)

<br>

If this notebook is helpful to you please consider buying me a coffee via [ko-fi](https://ko-fi.com/pharmapsychotic) or following me on [twitter](https://twitter.com/pharmapsychotic) for more cool Ai stuff. 🙂

And if you're looking for more Ai art tools check out my [Ai generative art tools list](https://pharmapsychotic.com/tools.html).


In [ ]:
#@title Setup
import os, subprocess
!git clone https://github.com/seedmanc/clip-interrogator-average.git
!pip install clip-interrogator-average/

def setup():
    install_cmds = [
        ['pip', 'install', 'gradio'],
        #['pip', 'install', 'open_clip_torch']
        #['pip', 'install', 'clip-interrogator-average'],
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()


caption_model_name = 'blip2-2.7b' #@param ["blip-base", "blip-large", "blip2-2.7b", "blip2-flan-t5-xl"]
clip_model_name = 'ViT-L-14/openai' #@param ["ViT-L-14/openai", "ViT-H-14/laion2b_s32b_b79k", "ViT-g-14/laion2B-s34B-b88K"]

import gradio as gr
from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = clip_model_name
config.caption_model_name = caption_model_name
ci = Interrogator(config)



In [ ]:
from PIL import Image

def image_to_prompt(files, mode):
    images = [Image.open(i).convert('RGB') for i in files]
    ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    return [ci.interrogate(images), ci.interrogate_orthogonal_fast(images), ci.interrogate_negative(images)]

In [ ]:
#@title Image to prompt! 🖼️ -> 📝

def prompt_tab():
    with gr.Column():
        with gr.Row():
            images = gr.Files(label="Image",file_types=["image"])
            with gr.Column():
                button = gr.Button("Generate prompts")
                prompt = gr.Textbox(label="Prompt")
                orthprompt = gr.Textbox(label="Irrelevant")
                negprompt = gr.Textbox(label="Negative")
    button.click(image_to_prompt, inputs=[images], outputs=[prompt,orthprompt,negprompt])

with gr.Blocks() as ui:
    prompt_tab()

ui.launch( debug=True,show_error=True)